In [ ]:
!pip install opencv-python deepface transformers librosa torch tensorflow
!pip install deepface opencv-python-headless

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.path.exists("/content/drive/MyDrive/data/6038291_Woman_Young_3840x2160.mp4"))

True


In [ ]:
import cv2
from deepface import DeepFace
from collections import Counter

def detect_video_emotion(video_path, num_frames=100):
    """Detect dominant emotion from video frames using DeepFace."""
    emotions = []  # Store detected emotions for each frame
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, frame_count // num_frames)  # Sample frames evenly

    print("Analyzing video frames for emotions...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_number % step == 0:  # Process every 'step'-th frame
            try:
                analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

                # Adjust handling for the response format
                if isinstance(analysis, list):
                    # For newer DeepFace versions returning a list of results
                    emotions.append(analysis[0]['dominant_emotion'])
                else:
                    # For older versions returning a dictionary
                    emotions.append(analysis['dominant_emotion'])

                print(f"Frame {frame_number}: {emotions[-1]}")
            except Exception as e:
                print(f"Error analyzing frame {frame_number}: {e}")

    cap.release()

    # Calculate the most frequent emotion across all analyzed frames
    if emotions:
        dominant_emotion = Counter(emotions).most_common(1)[0][0]
        return dominant_emotion
    else:
        return "Unknown"

# Example Usage
video_path = "/content/drive/MyDrive/data/6038291_Woman_Young_3840x2160.mp4"  # Replace with the path to your video file
dominant_emotion = detect_video_emotion(video_path)
print(f"Dominant Emotion in Video: {dominant_emotion}")


25-03-11 15:11:11 - Directory /root/.deepface has been created
25-03-11 15:11:11 - Directory /root/.deepface/weights has been created
Analyzing video frames for emotions...
25-03-11 15:11:19 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 69.9MB/s]


Frame 2: neutral
Frame 4: neutral
Frame 6: neutral
Frame 8: neutral
Frame 10: neutral
Frame 12: neutral
Frame 14: neutral
Frame 16: neutral
Frame 18: neutral
Frame 20: neutral
Frame 22: neutral
Frame 24: neutral
Frame 26: neutral
Frame 28: neutral
Frame 30: neutral
Frame 32: neutral
Frame 34: neutral
Frame 36: neutral
Frame 38: neutral
Frame 40: neutral
Frame 42: neutral
Frame 44: neutral
Frame 46: neutral
Frame 48: neutral
Frame 50: neutral
Frame 52: neutral
Frame 54: neutral
Frame 56: neutral
Frame 58: neutral
Frame 60: neutral
Frame 62: neutral
Frame 64: neutral
Frame 66: neutral
Frame 68: neutral
Frame 70: neutral
Frame 72: happy
Frame 74: happy
Frame 76: happy
Frame 78: happy
Frame 80: fear
Frame 82: happy
Frame 84: sad
Frame 86: happy
Frame 88: happy
Frame 90: fear
Frame 92: happy
Frame 94: sad
Frame 96: happy
Frame 98: happy
Frame 100: fear
Frame 102: happy
Frame 104: sad
Frame 106: happy
Frame 108: happy
Frame 110: happy
Frame 112: happy
Frame 114: happy
Frame 116: sad
Frame 11

In [ ]:
!pip install speechbrain opensmile librosa torch transformers
!pip install speechbrain torch librosa opensmile
!pip install speechbrain torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 36.2 MB/s eta 0:00:00


In [ ]:

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import os
import glob
import pandas as pd
import IPython.display as ipd
import plotly.express as px
import scipy.io.wavfile
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import torch
import torchaudio
import librosa
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

# Load a fully trained emotion classification model
model_name = "audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)

# Load the classroom audio file
audio_path = "/content/drive/MyDrive/data/kids-laugh-45357.mp3"
waveform, sample_rate = torchaudio.load(audio_path)

# Ensure correct sampling rate (16kHz required)
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# Convert to numpy and normalize
waveform = waveform.numpy().flatten()

# Process the audio
inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)

# Perform emotion classification
with torch.no_grad():
    logits = model(**inputs).logits

# Get the predicted emotion label
predicted_index = torch.argmax(logits, dim=-1).item()
emotion_labels = ["neutral", "happy", "sad", "angry", "fear", "disgust", "surprise"]
predicted_emotion = emotion_labels[predicted_index]
print(f"Predicted Emotion in Audio: {predicted_emotion}")

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Emotion in Audio: sad


In [ ]:
def fuse_emotions(video_emotion, audio_emotion):
    """Fusion strategy for combining video & audio emotions."""

    if video_emotion == audio_emotion:
        return video_emotion  # If both are the same, take the emotion directly

    elif video_emotion in ['happy', 'neutral'] and audio_emotion in ['happy', 'neutral']:
        return 'positive'  # If both are positive emotions

    elif video_emotion in ['angry', 'sad'] or audio_emotion in ['angry', 'sad']:
        return 'negative'  # If any one is negative, prioritize it

    else:
        return 'neutral'  # Default fallback

# Example usage
final_classroom_emotion = fuse_emotions(dominant_emotion, predicted_emotion)
print(f"Final Classroom Feedback: {final_classroom_emotion}")


Final Classroom Feedback: negative
